In [1]:
import numpy as np

In [12]:
a = np.arange(100000, dtype=np.float64)
np.random.shuffle(a)

In [15]:
%timeit min(a)

11.2 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [16]:
%timeit np.min(a)

3.06 µs ± 142 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
%timeit min(1, 2)

157 ns ± 2.83 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [13]:
%timeit np.min([1, 2])

5.38 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
import fynance as fy

In [21]:
%timeit np.asarray(fy.features.momentums_cy.roll_min_cy_1d_2(a, w=10000))

918 ms ± 34.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit np.asarray(fy.features.momentums_cy.roll_min_cy_1d(a, w=10000))

1.21 s ± 32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
import pandas as pd
import numpy as np

from collections import defaultdict

def cummax(g, v):
    df1 = pd.DataFrame(g, columns=['group'])
    df2 = pd.DataFrame(v)
    df = pd.concat([df1, df2], axis=1)

    result = df.groupby('group').cummax()
    result = result.values
    return result


def transform(g, v):
    df1 = pd.DataFrame(g, columns=['group'])
    df2 = pd.DataFrame(v)
    df = pd.concat([df1, df2], axis=1)

    result = df.groupby('group').transform(lambda x: x.cummax())
    result = result.values
    return result

def itertuples(g, v):
    df1 = pd.DataFrame(g, columns=['group'])
    df2 = pd.DataFrame(v)
    df = pd.concat([df1, df2], axis=1)

    d = defaultdict(list)

    result = [np.nan] * len(g)

    def d_(g, v):
        d[g].append(v)
        if len(d[g]) > 1:
            d[g][-1] = tuple(max(a,b) for (a,b) in zip(d[g][-2], d[g][-1]))
        return d[g][-1]

    for row in df.itertuples(index=True):
        index = row[0]
        result[index] = d_(row[1], row[2:])

    result = np.asarray(result)
    return result

def numpy(g, v):
    d = defaultdict(list)

    result = [np.nan] * len(g)

    def d_(g, v):
        d[g].append(v)
        if len(d[g]) > 1:
            d[g][-1] = np.maximum(d[g][-2], d[g][-1])
        return d[g][-1]

    for i in range(len(g)):
        result[i] = d_(g[i], v[i])

    result = np.asarray(result)
    return result



LENGTH = 100000
g = np.random.randint(low=0, high=LENGTH/2, size=LENGTH).astype(np.float64)
v = np.random.rand(LENGTH, 40)

%timeit r1 = cummax(g, v)
# %timeit r2 = transform(g, v)
%timeit r3 = itertuples(g, v)
%timeit r4 = numpy(g, v)

247 ms ± 22.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.65 s ± 12.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
313 ms ± 31.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit np.asarray(fy.features.momentums_cy.roll_min_cy_1d_2(g, w=0))
%timeit np.asarray(fy.features.momentums_cy.roll_min_cy_1d(g, w=0))

115 µs ± 3.16 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
121 µs ± 6.48 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
